In [36]:
#Importando as bibliotecas
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import sqlite3
from tqdm import tqdm
import os
import random


In [37]:
def Random_user_agents(marketplace):

    #Criando a lista de user-agents
    User_agents_list = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36 OPR/85.0.4341.60',
                        'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.130 Safari/537.36',
                        'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36',
                        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36',
                        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.101 Safari/537.36',
                        'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36',
                        'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.65 Safari/537.36',
                        'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
                        'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36']

    #Escolhendo o user aleatório
    Choice_user_agent = random.choice(User_agents_list)

    headers = {'authority':'www.' + marketplace + '.com.br',
               'user-agent':Choice_user_agent}


    return headers

In [38]:
Links_Kabum = []
Urls_Kabum = []
Sellers_Kabum = []
Country_Kabum = []
Price_Kabum = []
SKU_Kabum = []
Title_Kabum = []
Installment_Kabum_quantidade = []
Installment_Kabum_valor_parcela = []
Installment_Kabum_valor_total = []

In [39]:
def getting_n_creating_kabum(brand):

    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "+")
    df['Product_Name'] = df['Product_Name'].str.replace("-", "+")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "+" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://www.kabum.com.br/busca?query=" + df['Urls']

    return df

In [40]:
Df_products_raw = getting_n_creating_kabum("Wacom")
Df_products_raw

,Brand,Product_Name,Product_Descrition,Urls,Urls_search
0,wacom,ctl472,ctl 472 - mesa digitalizadora one by wacom peq...,wacom+ctl472,https://www.kabum.com.br/busca?query=wacom+ctl472
1,wacom,ctl4100,ctl4100 - mesa digitalizadora wacom intuos sma...,wacom+ctl4100,https://www.kabum.com.br/busca?query=wacom+ctl...
2,wacom,ctl4100wle0,ctl4100wle0 - mesa digitalizadora wacom intuos...,wacom+ctl4100wle0,https://www.kabum.com.br/busca?query=wacom+ctl...
3,wacom,ctl4100wlk0,ctl4100wlk0 - mesa digitalizadora wacom intuos...,wacom+ctl4100wlk0,https://www.kabum.com.br/busca?query=wacom+ctl...
4,wacom,ctl6100wlk0,ctl6100wlk0 - mesa digitalizadora wacom intuos...,wacom+ctl6100wlk0,https://www.kabum.com.br/busca?query=wacom+ctl...
5,wacom,dtc133w0a,dtc133w0a - mesa digitalizadora wacom one crea...,wacom+dtc133w0a,https://www.kabum.com.br/busca?query=wacom+dtc...
6,wacom,pth460,pth460 - mesa digitalizadora wacom intuos pro ...,wacom+pth460,https://www.kabum.com.br/busca?query=wacom+pth460
7,wacom,dtk1660k0a,dtk1660k0a - display interativo wacom cintiq 1...,wacom+dtk1660k0a,https://www.kabum.com.br/busca?query=wacom+dtk...
8,wacom,pth660,pth660 - mesa digitalizadora wacom intuos pro ...,wacom+pth660,https://www.kabum.com.br/busca?query=wacom+pth660
9,wacom,ctl6100wle0,ctl6100wle0 - mesa digitalizadora wacom intuos...,wacom+ctl6100wle0,https://www.kabum.com.br/busca?query=wacom+ctl...


In [41]:
def search_links(url):
    global Links_Kabum

    time.sleep(20)

    Headers_Kabum = Random_user_agents('kabum')
    response = requests.get(url, headers=Headers_Kabum)
    html = response.text

    Soup = BeautifulSoup(html, 'html.parser')

    for a in Soup.find_all("a", href=True):
        Links_Kabum.append("https://www.kabum.com.br" + a['href'])

    Links_Kabum = [s for s in Links_Kabum if 'produto' in s]

In [42]:
for url in tqdm(Df_products_raw['Urls_search']):
    search_links(url)

100%|██████████| 10/10 [03:25<00:00, 20.56s/it]


In [43]:
def get_attributes(url):
    time.sleep(20)

    Headers_Kabum = Random_user_agents('kabum')
    response = requests.get(url, headers=Headers_Kabum)

    html = response.text

    Soup = BeautifulSoup(html, 'html.parser')

    #Titulo
    try:
        Title_Kabum.append(Soup.find("h1", attrs={'itemprop':'name'}).text)
    except:
        Title_Kabum.append("Erro")

    #Preço
    try:
        Price_Kabum.append(Soup.find("h4", attrs={'itemprop':'price'}).text)
    except:
        Price_Kabum.append("Erro")

    #Seller
    try:
        Sellers_Kabum.append(Soup.find('div', attrs={'id':'blocoValores'}).text)
    except:
        Sellers_Kabum.append("Erro")

    #Installment
    try:
        Installment_Kabum_quantidade.append(Soup.find(class_='cardParcels').text)
    except:
        Installment_Kabum_quantidade.append("Erro")


In [44]:
for url in tqdm(Links_Kabum):
    get_attributes(url)

100%|██████████| 102/102 [34:59<00:00, 20.58s/it]


In [54]:
def dataset_creation(urls, sellers, prices, installments, titles):
    df_raw = pd.DataFrame()

    Hoje = pd.to_datetime('today', errors='ignore').date()

    df_raw['URL'] = urls

    df_raw['DATE'] = Hoje

    df_raw['MARKETPLACE'] = 'Kabum'

    df_raw['SELLER'] = sellers
    df_raw['SELLER'] = df_raw['SELLER'].str.partition("Vendido e entregue por: ")[2].str.partition(" |")[0]

    df_raw['Installment_lixo'] = installments

    df_raw['PARCEL'] = df_raw['Installment_lixo'].str.partition("x")[0].str.extract('(\d+)')

    df_raw['INSTALLMENT'] = df_raw['Installment_lixo'].str.partition("R$")[2]
    df_raw['INSTALLMENT'] = df_raw['INSTALLMENT'].str.partition(" ")[0]
    df_raw['INSTALLMENT'] = df_raw['INSTALLMENT'].str.replace(",",".", regex=True)
    df_raw['INSTALLMENT'] = df_raw['INSTALLMENT'].str.replace("\xa0","", regex=False)

    #df_raw['INSTALLMENT'] = df_raw['INSTALLMENT'].astype('float')

    df_raw['PRODUCT'] = titles

    df_raw['PRICE'] = prices

    df_raw = df_raw[df_raw['PRICE'] != "Erro"]

    df_raw['PRICE'] = df_raw['PRICE'].str.replace("R$","", regex=False)
    df_raw['PRICE'] = df_raw['PRICE'].str.replace(".","", regex=False)
    df_raw['PRICE'] = df_raw['PRICE'].str.replace(",",".", regex=False)
    df_raw['PRICE'] = df_raw['PRICE'].str.replace("\xa0","", regex=False)
    df_raw['PRICE'] = df_raw['PRICE'].astype('float')

    df_raw['ID'] = df_raw['URL'].str.partition("produto/")[2].str.partition("/")[0]

    df_raw = df_raw[['DATE', 'URL', 'MARKETPLACE', 'SELLER', 'PRICE', 'PARCEL','INSTALLMENT','ID','PRODUCT']]

    df_raw = df_raw.drop_duplicates(subset='URL')

    return df_raw

In [55]:
Dataset_Kabum = dataset_creation(Links_Kabum, Sellers_Kabum, Price_Kabum, Installment_Kabum_quantidade, Title_Kabum)

In [56]:
Dataset_Kabum

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,ID,PRODUCT
0,2022-05-26,https://www.kabum.com.br/produto/93434/mesa-di...,Kabum,KaBuM!,199.90,9,26.13,93434,Mesa Digitalizadora One by Wacom CTL472 Pequena
1,2022-05-26,https://www.kabum.com.br/produto/134333/mesa-d...,Kabum,Inpower Informatica,198.81,8,27.61,134333,Mesa Digitalizadora One by Wacom CTL472 Pequena
2,2022-05-26,https://www.kabum.com.br/produto/173541/mesa-d...,Kabum,Oficina dos Bits,300.59,10,33.39,173541,Mesa Digitalizadora One by Wacom Pequena - CTL...
3,2022-05-26,https://www.kabum.com.br/produto/208497/mesa-d...,Kabum,PRIMETEK,307.58,10,30.75,208497,Mesa Digitalizadora Wacom CTL472 Pequena 2540 ...
4,2022-05-26,https://www.kabum.com.br/produto/237192/mesa-d...,Kabum,Loja Ibyte,465.88,10,46.58,237192,Mesa Digitalizadora One By Wacom Ctl472 Pequena
6,2022-05-26,https://www.kabum.com.br/produto/95567/mesa-di...,Kabum,KaBuM!,299.90,10,35.28,95567,"Mesa Digitalizadora Wacom Intuos, Pequena, Can..."
7,2022-05-26,https://www.kabum.com.br/produto/349488/mesa-d...,Kabum,e-spot,659.00,10,65.90,349488,Mesa Digitalizadora Wacom Intuos Pequena Ctl4100
8,2022-05-26,https://www.kabum.com.br/produto/121143/mesa-d...,Kabum,Inpower Informatica,335.70,10,37.29,121143,"Mesa Digitalizadora Wacom, Intuos, Pequena - C..."
9,2022-05-26,https://www.kabum.com.br/produto/190581/mesa-d...,Kabum,Bits & Bytes,994.23,10,110.47,190581,Mesa Digitalizadora Wacom CTL-4100WL 4096 Pont...
10,2022-05-26,https://www.kabum.com.br/produto/208495/mesa-d...,Kabum,PRIMETEK,549.34,10,54.93,208495,Mesa Digitalizadora Wacom CTL4100 Intuos Peque...


In [33]:
print(len(Price_Kabum))

102


In [35]:
Price_Kabum

['Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro',
 'Erro']

In [23]:
Dataset_Kabum

NameError: name 'Dataset_Kabum' is not defined

In [19]:
Dataset_Kabum.dtypes

DATE            object
URL             object
MARKETPLACE     object
SELLER          object
PRICE          float64
PARCEL          object
INSTALLMENT     object
ID              object
PRODUCT         object
dtype: object

In [57]:
Dataset_Kabum.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Kabum_Wacom.xlsx", index=False)